In [23]:
import numpy as np
import pandas as pd

In [24]:
import re
import string

In [25]:
import pickle

In [26]:
with open('static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

C:\Users\User\Desktop\Sentiment_Analysis_Project\env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [27]:
with open('static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()


In [28]:
vocab = pd.read_csv('static/model/vocabulary.txt', header = None)
tokens = vocab[0].tolist()

In [29]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [30]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [31]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split())) 
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split()if x not in sw ))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\User\AppData\Local\Temp\ipykernel_1988\784901763.py:6: SyntaxWarning: invalid escape sequence '\d'
  data["tweet"] = data["tweet"].str.replace('\d+', '', regex=True)


In [32]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)
    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)

    return vectorized_list_new

In [60]:
def get_predictions(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return "negative"
    else:
        return 'positive'

In [61]:
txt1 = "Awesome product. i love it its very good"
preprocessed_txt = preprocessing(txt1)
vectorizer_text = vectorizer(preprocessed_txt, tokens)
prediction1 = get_predictions(vectorizer_text)

In [62]:
prediction1

'positive'

In [58]:
txt2 = "Awesome product. i love it"
preprocessed_txt2 = preprocessing(txt2)
vectorizer_text2 = vectorizer(preprocessed_txt2, tokens)
prediction2 = model.predict(vectorizer_text2)

In [59]:
prediction2

array([0], dtype=int64)